In [2]:
# Import dependencies
import pandas as pd

In [41]:
# Create resource file location
purchase_data = ('Resources/purchase_data.csv')

In [23]:
# Create df of the purchase data
pd_df = pd.read_csv(purchase_data, index_col='Purchase ID')
pd_df.head()

# Check for null values in the data set
# pd_df.loc[:,:].isnull().sum()

# pd_df.dtypes

,SN,Age,Gender,Item ID,Item Name,Price
Purchase ID,,,,,,
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,Ithergue48,24,Male,92,Final Critic,4.88
3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,Iskosia90,23,Male,131,Fury,1.44


In [8]:
# Displays all unique players to make purchases
player_count = pd_df['SN'].nunique()

# DataFrame to display player count
players = pd.DataFrame([player_count], columns=['Total Players'])
players

,Total Players
0,576


In [43]:
# Calculate item purchase metrics
num_item = pd_df['Item ID'].nunique() # Get number of unique items purchased
avg_price = '${0:,.2f}'.format(pd_df['Price'].mean()) # Get the average price per item
tot_purchases = pd_df['SN'].count() # Get the total number of purchases
tot_rev = '${0:,.2f}'.format(pd_df['Price'].sum()) # Get the total revenue for the purchases

# Create a summary table for the Purchasing Analysis
purchase_df = pd.DataFrame({'Number of Unique items':[num_item],
                            'Average Price':[avg_price],
                            'Number of Purchases':[tot_purchases],
                            'Total Revenue':[tot_rev]},
                          )
purchase_df

,Number of Unique items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [42]:
# Create new df to group numbers by gender
# Deleting dupes in the player SN to get a unique number of players in the group by
new_gender_df = pd_df.drop_duplicates('SN').groupby('Gender')

# Counting number of players by Gender using player SN
gender_count = new_gender_df['SN'].count()

# New DataFrame with a total count column by gender
gender_df = pd.DataFrame({'Total Count':gender_count})

# Calculating total number of players for percentage calc
total_count = gender_df['Total Count'].sum()

# Creating new column for percentage calc
gender_df['Percentage of Players'] = gender_df['Total Count']/total_count
gender_df['Percentage of Players'] = gender_df['Percentage of Players'].map('{0:.2%}'.format)

# Sorting gender counts by Total
gender_counts = gender_df.sort_values('Total Count', ascending=False)
gender_counts

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [39]:
# Calculating purchases by gender
# Sorting the DataFrame just by gender for total purchase
purch_df = pd_df.groupby('Gender')

# Calculating purchase metrics by gender
purch_count = purch_df['SN'].count() # Counting total purchases by gender using player SN
avg_purch_price = purch_df['Price'].mean() # Average Purchase price
tot_purch_val = purch_df['Price'].sum() # Total Purchase Value
avg_per = tot_purch_val/gender_count_uni # Average purchase per person/gender

# Generating DataFrame purchase stats by geneder
gender_purchase = pd.DataFrame({'Purchase Count':purch_count,
                                'Average Purchase Price':avg_purch_price,
                                'Total Purchase Value':tot_purch_val,
                                'Average Total Purchase per Person':avg_per}
                              )

gender_purchase = gender_purchase.style.format({'Average Purchase Price':'${0:,.2f}', 
                                                'Total Purchase Value':'${0:,.2f}', 
                                                'Average Total Purchase per Person':'${:.02f}'}
                                              )
gender_purchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [76]:
# Making bins to create age brackets
max_age = pd_df['Age'].max()
min_age = 10 # Min age value set by desired bin categories
size = (max_age - min_age)/7 # sets equal sized bins

bins = [0] # Initiates with a zero for lower limit
for i in range (0,8):
    bins.append((min_age-1) + (size*i)) # min_age - 1 since first bin is <10, not <=10
if bins[8] < max_age:
    bins[8] = max_age # setting upper bin value to include max age of 45 (bin would otherwise include up to 44 only)

labels = ['<10',
          '10-14',
          '15-19',
          '20-24',
          '25-29',
          '30-34',
          '35-39',
          '40+']

# Creating a new column to place player ages into brackets
pd_df['Age Summary'] = pd.cut(pd_df['Age'], bins, labels=labels, include_lowest=True)

# Dataframe to split out unique number of players into age categories
age_df_uni = pd_df.drop_duplicates('SN').groupby('Age Summary')

# Calculating total # of players in each age bracket
age_count_uni = age_df_uni['SN'].count()

# Dataframe to include total count
age_count = pd.DataFrame({'Total Count':age_count_uni})

# Calculating total # of players for percentage
total_age_count = age_count['Total Count'].sum()

# Adding column to new DataFrame to include percentage of total for each age group
age_count['Percentage of Players'] = age_count['Total Count']/total_age_count
age_count['Percentage of Players'] = age_count['Percentage of Players'].map('{0:.2%}'.format)
age_count

,Total Count,Percentage of Players
Age Summary,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [77]:
# Calculating purchasing by age group
age_df = pd_df.groupby('Age Summary')

# Calculating spend metrics by age group
age_pur_cnt = age_df['SN'].count() # Total purchases by age group
age_pur_avg = age_df['Price'].mean() # Average Purchase Price
age_pur_tot = age_df['Price'].sum() # Total Purchase Value
avg_pur_tot = age_pur_tot/age_count_uni # Average Total Purchase per Perspn

# DataFrame of purchase stats by age group
age_dem_pur = pd.DataFrame({'Purchase Count':age_pur_cnt,
                            'Average Purchase Price':age_pur_avg,
                            'Total Purchase Value':age_pur_tot,
                            'Avg Total Purchase per Person':avg_pur_tot}
                          )

age_dem_pur['Average Purchase Price'] = age_dem_pur['Average Purchase Price'].map('${0:,.2f}'.format)
age_dem_pur['Total Purchase Value'] = age_dem_pur['Total Purchase Value'].map('${0:,.2f}'.format)
age_dem_pur['Avg Total Purchase per Person'] = age_dem_pur['Avg Total Purchase per Person'].map('${0:,.2f}'.format)
age_dem_pur

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Summary,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [78]:
# Creating Top Spender by total purchase value
top_spend_df = pd_df.groupby('SN')

# Calculating spend metrics
top_spend_count = top_spend_df['SN'].count() # Total Purchases by player
top_spend_mean = top_spend_df['Price'].mean() # Average purchase price
top_spend_total = top_spend_df['Price'].sum() # Total purchase value

# DataFrame to record purchase stats by player
top_spend = pd.DataFrame({'Purchase Count':top_spend_count,
                          'Average Purchase Price':top_spend_mean,
                          'Total Purchase Value':top_spend_total}
                        )

top_spenders = top_spend.sort_values('Total Purchase Value', ascending=False) # Sort by Total Purchase Value
top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map('${0:,.2f}'.format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${0:,.2f}'.format)
top_spenders.head(5) # Looking for top 5

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [79]:
# Creating Popular Items by total purchase count
# DataFrame to bring in item info
pop_df = pd_df.loc[:, ['Item ID',
                       'Item Name',
                       'Price']
                  ].groupby(['Item ID','Item Name'])

# Calculating Price Metrics
pop_count = pop_df['Price'].count() # Total purchases by Item
pop_total = pop_df['Price'].sum() # Total purchase value by item
pop_price = pop_df['Price'].mean() # Avg purchase price of each item

# DataFrame to print top item metrics
pop_agg = pd.DataFrame({'Purchase Count':pop_count,
                        'Item Price':pop_price,
                        'Total Purchase Value':pop_total}
                      )

pop_items = pop_agg.sort_values('Purchase Count', ascending=False) # Sorting df by most purchased items
pop_items['Item Price'] = pop_items['Item Price'].map('${0:,.2f}'.format)
pop_items['Total Purchase Value'] = pop_items['Total Purchase Value'].map('${0:,.2f}'.format)
pop_items.head(5) # Only need top 5

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [80]:
# Sorting the above table to display most profitable item
pop_items = pop_agg.sort_values('Total Purchase Value', ascending=False)
pop_items['Item Price'] = pop_items['Item Price'].map('${0:,.2f}'.format)
pop_items['Total Purchase Value'] = pop_items['Total Purchase Value'].map('${0:,.2f}'.format)
pop_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
